In [ ]:
"""
Making figures for pressure and efield plots
"""
import numpy as np

import complete_edl as edl
import constants as C

%matplotlib widget

PH_WERT = 13
POTENTIALS_V_RHE = np.linspace(-1, 0, 100)
POTENTIALS = POTENTIALS_V_RHE - C.AU_PZC_SHE_V - 59e-3 * PH_WERT
DEFAULT_GAMMA = 4
DEFAULT_CONC = 1e-2  # 10 mM
GAMMA_RANGE = [3, 4, 5, 6]
CONC_RANGE = [10e-3, 100e-3, 1000e-3]

p_au_gamma = np.zeros((len(GAMMA_RANGE), len(POTENTIALS)))
e_au_gamma = np.zeros((len(GAMMA_RANGE), len(POTENTIALS)))
p_si_gamma = np.zeros((len(GAMMA_RANGE), len(POTENTIALS)))
e_si_gamma = np.zeros((len(GAMMA_RANGE), len(POTENTIALS)))

# Gamma sweep
for i, gamma in enumerate(GAMMA_RANGE):
    gold = edl.Aqueous(DEFAULT_CONC, gamma, DEFAULT_GAMMA, DEFAULT_GAMMA, 3)
    gold_sol = gold.potential_sweep(POTENTIALS, p_h=PH_WERT)
    p_au_gamma[i, :] = gold_sol["pressure"]
    e_au_gamma[i, :] = gold_sol["efield"]

    ins = edl.Aqueous(DEFAULT_CONC, gamma, DEFAULT_GAMMA, DEFAULT_GAMMA, 3)
    ins_sol = ins.insulator_spatial_profiles(p_h=PH_WERT, tol=1e-2)
    p_si_gamma[i, :] = np.ones(POTENTIALS.shape) * ins_sol["pressure"][0]
    e_si_gamma[i, :] = np.ones(POTENTIALS.shape) * ins_sol["efield"][0]

p_au_conc = np.zeros((len(CONC_RANGE), len(POTENTIALS)))
p_si_conc = np.zeros((len(CONC_RANGE), len(POTENTIALS)))
e_au_conc = np.zeros((len(CONC_RANGE), len(POTENTIALS)))
e_si_conc = np.zeros((len(CONC_RANGE), len(POTENTIALS)))

# Conc sweep
for i, conc in enumerate(CONC_RANGE):
    gold = edl.Aqueous(conc, DEFAULT_GAMMA, DEFAULT_GAMMA, DEFAULT_GAMMA, 3)
    gold_sol = gold.potential_sweep(POTENTIALS, p_h=PH_WERT)
    p_au_conc[i, :] = gold_sol["pressure"]
    e_au_conc[i, :] = gold_sol["efield"]

    ins = edl.Aqueous(conc, DEFAULT_GAMMA, DEFAULT_GAMMA, DEFAULT_GAMMA, 3)
    ins_sol = ins.insulator_spatial_profiles(p_h=PH_WERT, tol=1e-2)
    p_si_conc[i, :] = np.ones(POTENTIALS.shape) * ins_sol["pressure"][0]
    e_si_conc[i, :] = np.ones(POTENTIALS.shape) * ins_sol["efield"][0]


In [ ]:
import pandas as pd 
p_au_gamma_df = pd.DataFrame(
    data=p_au_gamma.T / 1e5, 
    index=POTENTIALS_V_RHE, 
    columns=[f'P / bar, Au, gamma = {gamma}' for gamma in GAMMA_RANGE],
)
e_au_gamma_df = pd.DataFrame(
    data=e_au_gamma.T * 1e-9, 
    index=POTENTIALS_V_RHE, 
    columns=[f'E / V/nm, Au, gamma = {gamma}' for gamma in GAMMA_RANGE],
)
p_si_gamma_df = pd.DataFrame(
    data=p_si_gamma.T / 1e5, 
    index=POTENTIALS_V_RHE, 
    columns=[f'P / bar, Ins, gamma = {gamma}' for gamma in GAMMA_RANGE],
)
e_si_gamma_df = pd.DataFrame(
    data=e_si_gamma.T * 1e-9, 
    index=POTENTIALS_V_RHE, 
    columns=[f'E / V/nm, Ins, gamma = {gamma}' for gamma in GAMMA_RANGE],
)
p_au_conc_df = pd.DataFrame(
    data=p_au_conc.T / 1e5, 
    index=POTENTIALS_V_RHE, 
    columns=[f'P / bar, Au, conc = {conc*1e3:.0f}' for conc in CONC_RANGE],
)
p_si_conc_df = pd.DataFrame(
    data=p_si_conc.T / 1e5, 
    index=POTENTIALS_V_RHE, 
    columns=[f'P / bar, Ins, conc = {conc*1e3:.0f}' for conc in CONC_RANGE],
)
e_au_conc_df = pd.DataFrame(
    data=e_au_conc.T * 1e-9, 
    index=POTENTIALS_V_RHE, 
    columns=[f'E / V/nm, Au, conc = {conc*1e3:.0f}' for conc in CONC_RANGE],
)
e_si_conc_df = pd.DataFrame(
    data=e_si_conc.T * 1e-9, 
    index=POTENTIALS_V_RHE, 
    columns=[f'E / V/nm, Ins, conc = {conc*1e3:.0f}' for conc in CONC_RANGE],
)

pressure_df = p_au_conc_df.join(p_si_conc_df, how='left').join(p_au_gamma_df, how='left').join(p_si_gamma_df, how='left')
pressure_df.index.name='Potential vs. RHE at pH 13 / V'
pressure_df.to_csv('pressure_data.csv')

In [ ]:
efield_df = e_au_conc_df.join(e_si_conc_df, how='left').join(e_au_gamma_df, how='left').join(e_si_gamma_df, how='left')
efield_df.index.name='Potential vs. RHE at pH 13 / V'
efield_df.to_csv('efield_data.csv')